In [1]:
import pandas as pd
impressions = pd.read_csv("impressions_one_hour.csv")

In [2]:
# drops "unnecessary" columns
impressions_cleaned = impressions.drop(columns=['Unnamed: 0', 'log_entry_time', 'partner_id', 
                                                             'private_contract_id', 'creative_id', 'ad_format',
                                                             'frequency', 'supply_vendor_publisher_id', 'deal_id',
                                                             'site', 'referrer_categories_list', 'fold_position',
                                                             'user_hour_of_week', 'ip_address', 'td_id', 'metro', 
                                                             'recency', 'language_code', 'media_cost', 
                                                             'fee_feature_cost', 'data_usage_total_cost', 
                                                             'ttd_cost_in_usd', 'partner_cost_in_usd', 
                                                             'advertiser_cost_in_usd', 'device_id', 
                                                             'processed_time', 'rendering_context', 
                                                             'temperature_in_celsius_name', 
                                                             'temperature_bucket_start_in_celsius_name', 
                                                             'temperature_bucket_end_in_celsius_name', 
                                                             'impression_placement_id', 'file_date', 
                                                             'advertiser_id'])
impressions_cleaned.head()

,impression_id,campaign_id,ad_group_id,audience_id,supply_vendor,user_agent,country,region,city,device_type,os_family,os,browser,latitude,longitude,zip_code,device_make,device_model,carrier_id
0,57f3aa7a-b276-47f7-a289-c78177691746,5txgufo,nudgj9z,8jx7gvs,mopub,Mozilla/5.0 (Linux; Android 8.1.0; Redmi S2 Bu...,Spain,A Coruña,Sada,4,6,154,15,43.35,-8.25,15160,Xiaomi,Redmi S2,577.0
1,2ecfdbff-b25a-446f-af7c-ffd62baefb02,wwvbnz8,y7xgocb,lzyn28h,mopub,Mozilla/5.0 (Linux; Android 6.0.1; SM-J500M Bu...,Brazil,Santa Catarina,São Miguel do Oeste,4,6,136,15,-26.71,-53.51,89900,Samsung,SM-J500M,503.0
2,b6e82c65-2521-45fe-94d5-6c9d37060e84,vh2uh6e,8z30tgg,8w2jhha,mopub,Mozilla/5.0 (iPhone; CPU iPhone OS 13_5_1 like...,France,Cote d'Or,Aiserey,4,5,168,15,47.17,5.16,21110,Apple,iPhone,407.0
3,9d26efa9-0cef-48d6-9619-4e73ce0a4829,vh2uh6e,8z30tgg,8w2jhha,google,Mozilla/5.0 (iPhone; CPU iPhone OS 13_6_1 like...,France,Seine-Maritime,Rouen,4,5,169,15,49.39,1.02,76000,Apple,iPhone,NaN
4,364a7c77-96ef-473c-9675-5e8ad2a0dbc1,f6l0r6n,3l27ron,8jx7gvs,google,Mozilla/5.0 (Linux; Android 10; SM-G970F Build...,Germany,Bavaria,Seubersdorf,4,6,101,15,49.16,11.62,92358,Samsung,SM-G970F,NaN


In [3]:
brazil_impressions = impressions_cleaned.copy()[impressions_cleaned['country'] == 'Brazil']
brazil_impressions.dropna(subset=['city'], inplace=True)
brazil_impressions["city"].value_counts()

São Paulo             12855
Rio de Janeiro         9027
Jardim Sao Paulo       5390
Brasília               3355
Salvador               3152
                      ...  
Rio de Contas             1
Carmo da Mata             1
Rondolandia               1
Sao Miguel do Anta        1
Lagoa do Carro            1
Name: city, Length: 3415, dtype: int64

## Population Distribution Comparison

In [5]:
ad_brazil_cities = pd.DataFrame(brazil_impressions['city'].value_counts(normalize=True)).reset_index()
ad_brazil_cities.rename(columns={"index": "name", "city": "ad_frequency"}, inplace=True)

In [6]:
brazil_city_pops = pd.read_csv('brazil_pop_by_city.csv')
brazil_city_pops = brazil_city_pops.drop(columns=['location'])
total_pop = brazil_city_pops['pop'].sum()
brazil_city_pops['proportion'] = brazil_city_pops['pop']/total_pop
brazil_city_pops = brazil_city_pops.drop(columns=['pop'])

ad_brazil_cities.set_index('name', inplace=True)
brazil_city_pops.set_index('name', inplace=True)
ad_brazil_cities

joined = brazil_city_pops.join(ad_brazil_cities)
joined.sort_values(by='ad_frequency', ascending=False)[:100]

,proportion,ad_frequency
name,,
Rio de Janeiro,0.058008,0.063662
Salvador,0.026115,0.022229
Porto Alegre,0.013219,0.021468
Manaus,0.015391,0.021439
Curitiba,0.016548,0.020530
...,...,...
Cubatao,0.001140,0.000959
Teresopolis,0.001194,0.000945
Francisco Morato,0.001654,0.000931


## Device Comparison
i.e. samsung, motorola, apple, etc.

In [32]:
brazil_impressions["device_make"].value_counts()
ad_device_data_brazil = pd.DataFrame(brazil_impressions["device_make"].value_counts(normalize=True)).reset_index()
ad_device_data_brazil.rename(columns={"index": "device_make", "device_make": "ad_frequency"}, inplace=True)
ad_device_data_brazil = ad_device_data_brazil.set_index("device_make")
len(ad_device_data_brazil)

#https://gs.statcounter.com/vendor-market-share/mobile-tablet-console/brazil/#monthly-201909-202009
#make sure its all devices, not just mobile

brazil_device_dist = pd.read_csv("brazil_all_device.csv")

distribution_2020 = brazil_device_dist[brazil_device_dist["Date"]=='2020-09'].iloc[0, 1:]/100
brazil_mobile_devices = pd.DataFrame(distribution_2020).rename(columns={12:'true frequency'})
brazil_mobile_devices.index = brazil_mobile_devices.index.rename("device_make")
len(brazil_mobile_devices)

joined_devices = brazil_mobile_devices.join(ad_device_data_brazil)
joined_devices

,true frequency,ad_frequency
device_make,,
Samsung,0.4458,0.400387
Motorola,0.2122,0.224221
Apple,0.1474,0.208224
Xiaomi,0.0789,0.053044
LG,0.0656,0.068222
Asus,0.0217,0.018933
Unknown,0.009,0.000508
Lenovo,0.0043,0.004050
Sony,0.0027,0.000974


In [33]:
brazil_impressions['device_type'].value_counts()

4    137245
3      4550
Name: device_type, dtype: int64

# Browsers
### i.e. Chrome, Safari, Firefox

## Browsers: All Devices

In [34]:
#https://gs.statcounter.com/browser-market-share/all/brazil
#distribution of all browsers 

brazil_browser = pd.read_csv('brazil_browser.csv')
brazil_browser_2020 = brazil_browser[brazil_browser["Date"]=='2020-09'].iloc[0, 1:]/100
brazil_browser_2020 = pd.DataFrame(brazil_browser_2020).rename(columns={12:'actual dist'})
brazil_browser_2020.index.name = 'browser_name'

ad_brazil_browser = pd.DataFrame(brazil_impressions['browser'].value_counts(normalize=True)).reset_index()
ad_brazil_browser.rename(columns={"index": "browser_name", "browser": "browser_frequency"}, inplace=True)
browser_map = {1: 'other', 7: 'Safari', 8: 'Opera', 6: 'Chrome', 15:'other' }
ad_brazil_browser['browser_name'] = ad_brazil_browser['browser_name'].map(browser_map)
ad_brazil_browser = ad_brazil_browser.set_index('browser_name')

ad_brazil_browser.join(brazil_browser_2020)

,browser_frequency,actual dist
browser_name,,
Chrome,0.018851,0.6805
Opera,0.000078,0.0221
Safari,0.000719,0.1072
other,0.979689,NaN
other,0.000663,NaN


## Browser: Mobile

In [35]:
#https://gs.statcounter.com/browser-market-share/mobile/brazil/#monthly-201909-202009
#browser on mobile devices 

browser_map = {1: 'other', 7: 'Safari', 8: 'Opera', 6: 'Chrome', 15:'other' }

brazil_mobile_browser = pd.read_csv('brazil_mobile_browser.csv')
brazil_mobile_browser = brazil_mobile_browser[brazil_mobile_browser["Date"]=='2020-09'].iloc[0, 1:]/100
brazil_mobile_browser = pd.DataFrame(brazil_mobile_browser).rename(columns={12:'actual dist'})
brazil_mobile_browser.index.name = 'browser_name'

mobile = brazil_impressions[brazil_impressions['device_type']==4]

ad_brazil_mobile_browser = pd.DataFrame(mobile['browser'].value_counts(normalize=True)).reset_index()
ad_brazil_mobile_browser.rename(columns={"index": "browser_name", "browser": "browser_frequency"}, inplace=True)
ad_brazil_mobile_browser['browser_name'] = ad_brazil_mobile_browser['browser_name'].map(browser_map)
ad_brazil_mobile_browser = ad_brazil_mobile_browser.set_index('browser_name')

ad_brazil_mobile_browser.join(brazil_mobile_browser)

,browser_frequency,actual dist
browser_name,,
Chrome,0.016707,0.7897
Opera,0.000073,0.0051
Safari,0.000372,0.1295
other,0.982178,NaN
other,0.000670,NaN


## Browser: Tablet

In [36]:
# https://gs.statcounter.com/browser-market-share/tablet/brazil/#monthly-201909-202009
# browser on tablet distrbution comparison

browser_map = {1: 'other', 7: 'Safari', 8: 'Opera', 6: 'Chrome', 15:'other' }

brazil_tablet_browser = pd.read_csv('brazil_tablet_browser.csv')
brazil_tablet_browser = brazil_tablet_browser[brazil_tablet_browser["Date"]=='2020-09'].iloc[0, 1:]/100
brazil_tablet_browser = pd.DataFrame(brazil_tablet_browser).rename(columns={12:'actual dist'})
brazil_tablet_browser.index.name = 'browser_name'

tablet = brazil_impressions[brazil_impressions['device_type']==3]

ad_brazil_tablet_browser = pd.DataFrame(tablet['browser'].value_counts(normalize=True)).reset_index()
ad_brazil_tablet_browser.rename(columns={"index": "browser_name", "browser": "browser_frequency"}, inplace=True)
ad_brazil_tablet_browser['browser_name'] = ad_brazil_tablet_browser['browser_name'].map(browser_map)
ad_brazil_tablet_browser = ad_brazil_tablet_browser.set_index('browser_name')

ad_brazil_tablet_browser.join(brazil_tablet_browser)

,browser_frequency,actual dist
browser_name,,
Chrome,0.083516,0.535
Opera,0.000220,0.0107
Safari,0.011209,0.2892
other,0.904615,NaN
other,0.000440,NaN


## Device Type: General
### i.e. Desktop vs mobile vs Tablet Distribution

In [37]:
#https://gs.statcounter.com/platform-market-share/desktop-mobile-tablet/brazil
#distribution of device types 

device_type_map = {1: 'Other', 2: 'PC', 3: 'Tablet', 4:'Mobile', 5:'Roku', 6:'ConnectedTV'}
brazil_impressions['device_type'].map(device_type_map)
ad_device_type = brazil_impressions['device_type'].map(device_type_map).value_counts(normalize=True)

device_type = pd.read_csv('brazil_desktop_v_mobile.csv')
device_type_distribution_2020 = device_type[device_type["Date"]=='2020-09'].iloc[0, 1:]/100
device_type_distribution_2020 = pd.DataFrame(device_type_distribution_2020)
device_type_distribution_2020 = device_type_distribution_2020.rename(columns={12: 'actual_device_dist'})

device_type_distribution_2020.join(ad_device_type)

,actual_device_dist,device_type
Desktop,0.7085,NaN
Mobile,0.2868,0.967911
Tablet,0.0047,0.032089


# Operating Systems

## Operating System: All


In [38]:
#https://gs.statcounter.com/os-market-share/all/brazil/#monthly-201909-202009
#all operating system comparisons

os_map = {1: 'other', 2:'Windows', 3:'OS X', 4: 'Linux', 5:'iOS', 6:'Android', 7:'Windows Phone'}

ad_os = pd.DataFrame(brazil_impressions['os_family'].map(os_map).value_counts(normalize=True))
ad_os.index.name = 'name'
ad_os = ad_os.rename(columns={'os_family':'ad_mobile_os'})

brazil_os = pd.read_csv('brazil_os_general.csv')
brazil_os = brazil_os[brazil_os["Date"]=='2020-09'].iloc[0, 1:]/100
brazil_os = pd.DataFrame(brazil_os).rename(columns={12:'actual dist'})
brazil_os.index.name = 'name'
brazil_os.join(ad_os)

,actual dist,ad_mobile_os
name,,
Windows,0.6313,NaN
Android,0.2476,0.791495
OS X,0.0655,NaN
iOS,0.043,0.208505
Linux,0.0065,NaN
Unknown,0.0043,NaN
Chrome OS,0.0006,NaN
Playstation,0.0004,NaN
Samsung,0.0004,NaN


## Operating System: Mobile Device

In [39]:
#https://gs.statcounter.com/os-market-share/mobile/brazil/#monthly-201909-202009
#operating system broken down by mobile device

os_map = {1: 'other', 2:'Windows', 3:'OS X', 4: 'Linux', 5:'iOS', 6:'Android', 7:'Windows Phone'}

mobile = brazil_impressions[brazil_impressions['device_type']==4]

ad_mobile_os = pd.DataFrame(mobile['os_family'].map(os_map).value_counts(normalize=True))
ad_mobile_os.index.name = 'name'
ad_mobile_os = ad_mobile_os.rename(columns={'os_family':'ad_mobile_os'})

brazil_os_mobile = pd.read_csv('brazil_os_for_mobile.csv')
brazil_os_mobile = brazil_os_mobile[brazil_os_mobile["Date"]=='2020-09'].iloc[0, 1:]/100
brazil_os_mobile = pd.DataFrame(brazil_os_mobile).rename(columns={12:'actual dist'})
brazil_os_mobile.index.name = 'name'
brazil_os_mobile.join(ad_mobile_os)

,actual dist,ad_mobile_os
name,,
Android,0.854,0.802368
iOS,0.1438,0.197632
Samsung,0.0014,NaN
Windows,0.0004,NaN
Unknown,0.0001,NaN
Series 40,0,NaN
Other,0.0002,NaN


## Operating System: Tablet

In [40]:
#https://gs.statcounter.com/os-market-share/tablet/brazil/#monthly-201909-202009
#operating system broken down by tablet

os_map = {1: 'other', 2:'Windows', 3:'OS X', 4: 'Linux', 5:'iOS', 6:'Android', 7:'Windows Phone'}

tablet = brazil_impressions[brazil_impressions['device_type']==3]

ad_mobile_os = pd.DataFrame(tablet['os_family'].map(os_map).value_counts(normalize=True))
ad_mobile_os.index.name = 'name'
ad_mobile_os = ad_mobile_os.rename(columns={'os_family':'ad_mobile_os'})

brazil_os_mobile = pd.read_csv('brazil_tablet_os.csv')
brazil_os_mobile = brazil_os_mobile[brazil_os_mobile["Date"]=='2020-09'].iloc[0, 1:]/100
brazil_os_mobile = pd.DataFrame(brazil_os_mobile).rename(columns={12:'actual dist'})
brazil_os_mobile.index.name = 'name'
brazil_os_mobile.join(ad_mobile_os)

,actual dist,ad_mobile_os
name,,
Android,0.6167,0.463516
iOS,0.3824,0.536484
Linux,0.0004,NaN
Windows,0.0002,NaN
BlackBerry OS,0.0002,NaN
webOS,0,NaN
Other,0,NaN
